In [ ]:
# @title Libraries
import os
import tensorflow as tf
from google.colab import drive
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import math
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor,ExtraTreesRegressor
import numpy as np
import statistics
from sklearn.feature_selection import VarianceThreshold
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from hyperopt import fmin, tpe, hp, Trials
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import r_regression, f_regression, mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import PredictionErrorDisplay
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
import time
import statistics
from itertools import product
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
# from mlxtend.feature_selection import FSSearch
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from itertools import product
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Variable
# @title
variable_name = "" # @param {type:"string"}
################ PARAMETERS ####################

DATASET = ['SNMMI_CHALLENGE_TRAINING_V22OCT2023_Update']
################################################
import warnings
result_path = '/content/drive/MyDrive/Challenge_SNMMI_2023/Results/Amirali & Nima/22 12 2023/'
def append_row(df, row):
    return pd.concat([
                df,
                pd.DataFrame([row], columns=row.index)]
          ).reset_index(drop=True)

In [ ]:
# @title Load data
df = pd.read_excel("/content/drive/MyDrive/Challenge_SNMMI_2023/Data/SNMMI_CHALLENGE_TRAINING_V22OCT2023_Update Oct 26.xlsx", index_col=0)
dftest = pd.read_excel("/content/drive/MyDrive/Challenge_SNMMI_2023/Data/SNMMI_CHALLENGE_TESTING_V01112023.xlsx", index_col = 0)

def convert_to_groups(number):
    if 0 <= number <= 12:
        return 1
    elif 12 < number <= 24:
        return 2
    elif 24 <number <=36:
        return 3
    else:
      return 4
# test data
X_mainL1_test = dftest

filtered_df_s = df[df['Event'] != 0]
target_column1 = filtered_df_s["Outcome_PFS"].apply(convert_to_groups)

Y_mainL1_train = target_column1
sup_indxs = Y_mainL1_train.index

X_mainL1_train = df.loc[sup_indxs].drop(columns=['Event', 'Outcome_PFS'])






filtered_df_ss = df[df['Event'] == 0]
target_column2 = filtered_df_ss["Outcome_PFS"].apply(convert_to_groups)

Y_mainL2_train = target_column2
sup_indxs2 = Y_mainL2_train.index

X_mainL2_train = df.loc[sup_indxs2].drop(columns=['Event', 'Outcome_PFS'])
X_mainL2_train = X_mainL2_train.append(X_mainL1_test)





<ipython-input-4-ba5f0fa7a5bc>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_mainL2_train = X_mainL2_train.append(X_mainL1_test)


In [ ]:
# @title Logestic Regression
class_name ='LR'
#Lib
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.exceptions import FitFailedWarning
warnings.filterwarnings("ignore", category=FitFailedWarning)

results1 = {"Regressor":[],
"Feature_Selection":[],
'Kfold':[],
'Scaler' :[],
'test_Accuracy':[],
'train_Accuracy': []
}
# Initialize and split the data outside the loop if it remains constant
X_train = X_mainL1_train
y_train = Y_mainL1_train
X_test = X_mainL1_test
# y_test = Y_mainL1_test

# imputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# Scaler
scaler = MinMaxScaler()

#pca
pca = PCA(n_components=2)

# Feature Selection
feat_selec = SelectKBest(f_regression, k=10)

# Import the model
clas = LogisticRegression()

# Pipeline
pipe = Pipeline(steps=[("imputer", imputer), ("STANDARD", scaler),("MINMAX", scaler),("feat_select",feat_selec), ("model", clas) ])

# Define hyperparameters to tune
param_grid = {
    'model__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'model__penalty': ['l1', 'l2'],
    'model__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag'],
    'model__max_iter': [100, 500, 1000]
}

#Define k_fold for Inner Loop
cv1 = KFold(n_splits=5, shuffle=True, random_state=0)

# Inner loop
grid_search = GridSearchCV(pipe, param_grid, cv=cv1, scoring='accuracy')


#Define k_fold for Outer Loop
cv2 = KFold(n_splits=5, shuffle=True, random_state=0)
# Outer loop
cv_results = cross_validate(grid_search, X_train, y_train, cv=cv2, scoring='accuracy', return_train_score=True)


#Accuracy
test_ac_mae = np.mean(cv_results['test_score'])
train_ac_mae = np.mean(cv_results['train_score'])

testD_ac_mae = np.std(cv_results['test_score'])
trainD_ac_mae = np.std(cv_results['train_score'])

# Make predictions on the training data
grid_search.fit(X_train, y_train)

X_train_unlabeled = X_mainL2_train
y_unlabeled_pred = grid_search.predict(X_train_unlabeled)


# Create a new dataset with the predicted labels and add it to the labeled data
new_X_labeled = np.vstack((X_train, X_train_unlabeled))
new_y_labeled = np.concatenate((y_train, y_unlabeled_pred))


cv_results = cross_validate(grid_search, new_X_labeled, new_y_labeled, cv=cv2, scoring= 'accuracy', return_train_score=True)


#Accuracy
test_ac_mae = np.mean(cv_results['test_score'])
train_ac_mae = np.mean(cv_results['train_score'])

testD_ac_mae = np.std(cv_results['test_score'])
trainD_ac_mae = np.std(cv_results['train_score'])





print(f'test_Accuracy:{test_ac_mae:.2f}±{testD_ac_mae:.2f}')
print(f'train_Accuracy:{train_ac_mae:.2f}±{trainD_ac_mae:.2f}')

results1["Regressor"].append(class_name)
results1['Kfold'].append(10)
results1["Feature_Selection"].append('fr')
results1["Scaler"].append(scaler)


results1['test_Accuracy'].append(f"{test_ac_mae:.2f}±{testD_ac_mae:.2f}")
results1['train_Accuracy'].append(f"{train_ac_mae:.2f}±{trainD_ac_mae:.2f}")




/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.44761905        nan 0.44761905 0.44761905
 0.44761905 0.44761905        nan        nan 0.44761905        nan
 0.44761905 0.44761905 0.44761905 0.44761905        nan        nan
 0.44761905        nan 0.44761905 0.44761905 0.44761905 0.44761905
        nan        nan 0.44761905        nan 0.44761905 0.44761905
 0.44761905 0.44761905        nan        nan 0.44761905        nan
 0.44761905 0.44761905 0.44761905 0.44761905        nan        nan
 0.44761905        nan 0.44761905 0.44761905 0.44761905 0.44761905
        nan        nan 0.44761905        nan 0.47428571 0.47428571
 0.51714286 0.47428571        nan        nan 0.44761905        nan
 0.47428571 0.47428571 0.51714286 0.47428571        nan        nan
 0.44761905        nan 0.47428571 0.47428571 0.51714286 0.47428571
        nan        nan 0.50285714        nan 0.5152381  

test_Accuracy:0.79±0.03
train_Accuracy:0.80±0.03


,0,1,2
0,0.736496,0.692442,0.448384
1,0.862515,0.856770,0.303104
2,0.903526,0.898828,0.746542
3,0.390755,0.173927,0.060701
4,0.562539,0.445347,0.267632
...,...,...,...
335,0.738306,0.699621,0.471289
336,0.697342,0.601998,0.533434
337,0.915524,0.912660,0.782514
338,0.604817,0.411817,0.200576


In [ ]:
predict = grid_search.predict_proba(X_test)
df_p = pd.DataFrame(predict)

df_p[2] = 1-df_p[0]-df_p[1]-df_p[2]
df_p[1] = 1-df_p[0]-df_p[1]
df_p[0] = 1-df_p[0]

df_p.drop(3, axis=1, inplace=True)

df = pd.DataFrame(results1)
df.to_excel(result_path+'LR_Classification.xlsx', index = False)

df_p.to_excel(result_path+'LR_Classification_Prediction.xlsx')